In [ ]:
import pandas as pd
from pathlib import Path
import os
from Bio import SeqIO
import glob

### Selecting the IDs that have all signatures common to the references and sorting them

This directory must contain:
- The file common_signatures_per_gene.csv generated by the interproscan_ref_summary.ipynb notebook;
- A directory containing the InterProScan output files for the database sequences.

The file common_signatures_per_gene.csv was modified to merge lic3A and lic3B into a single classification (lic3X), and to treat lst and cpsK as a single classification (lst). These changes were applied because these proteins share identical InterPro signatures.

In [ ]:
# === 1. Input paths ===
signatures_csv = "common_signatures_per_gene.csv"  # your file with relevant signatures
interpro_dir = Path("./interpro_outputs/")             # directory with InterProScan .tsv files
output_dir = Path("./filtered_sequences/")
output_dir.mkdir(exist_ok=True)

# === 2. Read the relevant signatures file ===
df_signatures = pd.read_csv(signatures_csv)
df_signatures['common_signatures'] = df_signatures['common_signatures'].astype(str)

# Dictionary: gene -> set of signatures
signatures_dict = (
    df_signatures.groupby('Gene')['common_signatures']
    .apply(set)
    .to_dict()
)

# === 3. Function to process an InterProScan file ===
def read_interproscan_tsv(file):
    columns = [
        "seq_id", "md5", "length", "analysis", "signature_accession",
        "signature_desc", "start", "end", "evalue", "status",
        "date", "ipr", "ipr_desc", "go", "pathway"
    ]
    try:
        df = pd.read_csv(file, sep="\t", header=None, names=columns, usecols=[0, 4])
        return df
    except Exception as e:
        print(f"Error reading {file}: {e}")
        return pd.DataFrame(columns=["seq_id", "signature_accession"])

# === 4. Combine all InterProScan files ===
dfs = []
for file in interpro_dir.glob("*.tsv"):
    df_tmp = read_interproscan_tsv(file)
    dfs.append(df_tmp)

df_interpro = pd.concat(dfs, ignore_index=True)
df_interpro.dropna(subset=["seq_id", "signature_accession"], inplace=True)

# === 5. Group signatures by sequence ===
signatures_per_seq = (
    df_interpro.groupby("seq_id")["signature_accession"]
    .apply(set)
    .to_dict()
)

# === 6. For each gene, find sequences that contain all signatures ===
for gene, reference_signatures in signatures_dict.items():
    matching_ids = [
        seq_id for seq_id, seq_signatures in signatures_per_seq.items()
        if reference_signatures.issubset(seq_signatures)
    ]
    
    # Save result
    output_path = output_dir / f"{gene}_sequences.txt"
    with open(output_path, "w") as f:
        for seq_id in matching_ids:
            f.write(seq_id + "\n")
    
    print(f"✅ {gene}: {len(matching_ids)} sequences saved to {output_path}")

### Organizing the filtered sequences in a summary

In [ ]:
# === Main paths ===
dir_ids = "./filtered_sequences"          # where the gene_sequences.txt files are located
dir_tsv = "./interpro_outputs/"            # where the InterProScan .tsv files are located

# List to accumulate the data
data = []

# === 1. Load all InterProScan outputs ===
# Read all TSVs into a single DataFrame
df_list = []
for file in os.listdir(dir_tsv):
    if file.endswith(".tsv"):
        path = os.path.join(dir_tsv, file)
        try:
            df_tmp = pd.read_csv(path, sep="\t", header=None, dtype=str)
            df_tmp = df_tmp[[0, 4]]  # Column 0 = seq_id, Column 4 = signature (IPRxxxxx)
            df_tmp.columns = ["seq_id", "signature"]
            df_list.append(df_tmp)
        except Exception as e:
            print(f"⚠️ Error reading {file}: {e}")

# Concatenate all InterProScan outputs into a single DataFrame
df_all = pd.concat(df_list, ignore_index=True)
print(f"✅ .tsv files combined: {len(df_all)} total rows")

# === 2. For each gene, retrieve the signatures ===
for file in os.listdir(dir_ids):
    if file.endswith("_sequences.txt"):
        gene = file.split("_sequences.txt")[0]
        ids_path = os.path.join(dir_ids, file)

        # Read sequence IDs (one per line)
        with open(ids_path) as f:
            ids = [line.strip() for line in f if line.strip()]

        # Filter the large dataframe by the sequences of this gene
        df_filtered = df_all[df_all["seq_id"].isin(ids)].copy()

        # Add gene column
        df_filtered["gene"] = gene

        data.append(df_filtered)

        print(f"🔹 {gene}: {len(df_filtered)} signatures found for {len(ids)} sequences")

# === 3. Merge everything and save ===
df_final = pd.concat(data, ignore_index=True)

# Reorder columns
df_final = df_final[["seq_id", "gene", "signature"]]

# Show first rows
print(df_final.head())

# === 4. (Optional) save to CSV ===
df_final.to_csv("filtered_seqs_summary.csv", index=False)

### Checking which sequences have signatures other than the reference ones

In [ ]:
# === 1. Read files ===
df_all = pd.read_csv("./filtered_seqs_summary.csv")
df_ref = pd.read_csv("./common_signatures_per_gene.csv")

# === 2. Ensure consistency and cleaning ===
# Clean column names
df_ref.columns = df_ref.columns.str.strip()
df_all.columns = df_all.columns.str.strip()

# Normalize column contents
for col in ["gene", "signature"]:
    if col in df_all.columns:
        df_all[col] = (
            df_all[col]
            .astype(str)
            .str.strip()
            .str.replace("\r", "", regex=False)
            .str.replace("\n", "", regex=False)
            .str.upper()
        )

if "Gene" in df_ref.columns:
    df_ref["Gene"] = (
        df_ref["Gene"]
        .astype(str)
        .str.strip()
        .str.replace("\r", "", regex=False)
        .str.replace("\n", "", regex=False)
        .str.upper()
    )

# Convert reference signatures into lists
df_ref["common_signatures"] = df_ref["common_signatures"].apply(
    lambda x: [s.strip().replace("\r", "").replace("\n", "").upper() for s in str(x).split(";") if s.strip()]
)

# === 3. Group signatures by sequence ===
seq_signatures = (
    df_all.groupby(["seq_id", "gene"])["signature"]
    .apply(lambda x: set(s.strip().replace("\r", "").replace("\n", "").upper() for s in x if s))
    .reset_index()
)

# === 4. Compare with the reference set ===
results = []

for _, row in seq_signatures.iterrows():
    seq_id = row["seq_id"]
    gene = row["gene"]
    sigs = row["signature"]

    # Reference rows for the same gene
    ref_row = df_ref[df_ref["Gene"] == gene]
    if ref_row.empty:
        continue  # skip if the gene has no reference

    # Merge all reference signature lists
    ref_sigs = set().union(*ref_row["common_signatures"])

    # Check if it contains all reference signatures AND has extras
    if ref_sigs.issubset(sigs) and len(sigs - ref_sigs) > 0:
        extra = sigs - ref_sigs
        results.append({
            "seq_id": seq_id,
            "gene": gene,
            "extra_signatures": ";".join(sorted(extra))
        })

# === 5. Save results ===
df_result = pd.DataFrame(results)
df_result.to_csv("seqs_with_extra_signatures.csv", index=False)

print(f"{len(df_result)} sequences have additional signatures.")
print(df_result.head())

In [ ]:
# THIS CODE HELPED ME DURING CURATION – BUT I NO LONGER USE IT

# Read the original output file
df = pd.read_csv("seqs_with_extra_signatures.csv")

# === Text normalization ===
df["gene"] = df["gene"].astype(str).str.strip().str.upper()
df["extra_signatures"] = (
    df["extra_signatures"]
    .astype(str)
    .str.strip()
    .str.upper()
    .str.replace("\r", "", regex=False)
    .str.replace("\n", "", regex=False)
)

# === Removal conditions ===

# 1️⃣ Remove COIL / MOBIDB-LITE (alone or together, any order)
cond1 = df["extra_signatures"].isin([
    "COIL",
    "MOBIDB-LITE",
    "COIL;MOBIDB-LITE",
    "MOBIDB-LITE;COIL"
])

# 2️⃣ Remove specific G3DSA when gene == LST
cond2 = (df["gene"] == "LST") & df["extra_signatures"].isin([
    "G3DSA:3.40.50.1110",
    "G3DSA:3.30.370.20"
])

# 3️⃣ Remove specific G3DSA when gene == LIC3X
cond3 = (df["gene"] == "LIC3X") & (df["extra_signatures"] == "G3DSA:3.90.1480.10")

# Remove PF13472 (GDSL-like Lipase/Acylhydrolase family)
cond4 = (df["gene"] == "NEUA") & df["extra_signatures"].isin(["PF13472"])

# Remove SSF53756 (UDP-Glycosyltransferase)
cond5 = df["extra_signatures"].str.contains(r"\bSSF53756\b", na=False, regex=True)

# Remove PS51257 (lipoprotein lipid attachment site profile)
cond6 = df["extra_signatures"].str.contains(r"\bPS51257\b", na=False, regex=True)

# Remove SSF52266 (SGNH hydrolases)
cond7 = df["extra_signatures"].str.contains(r"\bSSF52266\b", na=False, regex=True)

# Remove PF00535 (Glycosyl transferase family 2)
cond8 = df["extra_signatures"].str.contains(r"\bPF00535\b", na=False, regex=True)

# Remove PF01670
cond9 = df["extra_signatures"].str.contains(r"\bPF01670\b", na=False, regex=True)

# Remove CD00761
cond10 = df["extra_signatures"].str.contains(r"\bCD00761\b", na=False, regex=True)

# Combine everything
cond_total = cond1 | cond2 | cond3 | cond4 | cond5 | cond6 | cond7 | cond8 | cond9 | cond10

# === Apply filter ===
df_filtered = df[~cond_total]

# === Save result ===
df_filtered.to_csv("seqs_with_extra_signatures_filtered.csv", index=False)

# === Report ===
n_removed = len(df) - len(df_filtered)
print(f"{n_removed} rows were removed.")
print(f"{len(df_filtered)} remaining rows saved to 'seqs_with_extra_signatures_filtered.csv'.")

### Taking the dataframe seqs_with_extra_signatures.csv and filtering what will definitely be excluded

In [ ]:
# === 1. Read original file ===
file = "seqs_with_extra_signatures.csv"
df = pd.read_csv(file)

# === 2. Text normalization ===
df["gene"] = df["gene"].astype(str).str.strip().str.upper()
df["extra_signatures"] = (
    df["extra_signatures"]
    .astype(str)
    .str.strip()
    .str.upper()
)

# === 3. Define lists and conditions ===

signatures_to_remove = [
    "PF15632", "PF00903", "PF01973", "PF01370", "PF02826", "PF07883", "PF01029",
    "PF13411", "PF22725", "PF01408", "PF03737", "PF13561", "PF00389", "PF17836",
    "PF00005", "PF03102", "PF00483", "PF04545", "PF08240", "PF09861", "PF02558",
    "PF01583", "PS50007", "PS50008", "PS50175", "PF00459", "PF02737", "PF04932",
    "PS00126", "PS01319", "PS50890", "PS50893", "SSF51161", "CD04647", "PTHR36489", 
    "PTHR42781", "TIGR03584", "PF08282", "PF22352", "SSF56784", "PF06306", "SSF53335", 
    "PF01895", "SSF109755", "PS00211", "SSF52540", "PR01179", "SSF50621", "PF02784", 
    "SSF51419", "SFLDS00003", "SFLDG01136", "SFLDG01138", "TIGR01670", "CD01630",
    "PF00535", "SSF53448", "PF04101", "PF01670", "PF13302", "PF04101",
    "PF01501", "CD03801", "G3DSA:3.40.50.2000", "PF00571", "PF02350", 
    "PF14602", "CD10144", "PF14602", "PF08889", "SSF51735", "PF00364", 
    "PF15632", "SSF53335", "SSF51230", "PF13439", "SSF48452", "PS50287",
    "PF13946", "PF12895", "SSF81483", "TIGR03028", "SSF142984", "PF06251",
    "PS51257", "PS51318", "G3DSA:3.10.20.600", "PF01061", "PS00858",      
    "PF00535", "PS00430", "PS50180", "PF01469", "PS50889", "PS50832",
    "PF03448", "CD22265", "SSF103657", "PF13469", "PF05686", "SSF56784",
    "PF02706", "PR01868", "CD06530", "PS50096", "PF00756", "PF04230",
    "G3DSA:1.10.3130.20", "PS00307", "G3DSA:2.70.50.60", "G3DSA:1.20.5.340", 
    "PF01041", "PS51191", "PF04339", "SSF57997"   
]

# Helper function — converts signatures into a list and checks for presence
def has_inadequate_signature(signatures, forbidden):
    sig_list = [s.strip() for s in str(signatures).split(";")]
    return any(sig in sig_list for sig in forbidden)

# cond1: remove any row that contains at least one forbidden signature
cond1 = df["extra_signatures"].apply(lambda x: has_inadequate_signature(x, signatures_to_remove))

# cond2: neuA or neuS → remove if it contains sialyltransferase signatures
cond2 = (
    df["gene"].isin(["NEUA", "NEUS"]) &
    df["extra_signatures"].apply(lambda x: has_inadequate_signature(x, ["PF07922", "SSF102414", "PF11477"]))
)

# cond3: neuA → remove if it contains neuS signatures
cond3 = (
    (df["gene"] == "NEUA") &
    df["extra_signatures"].apply(lambda x: has_inadequate_signature(x, ["PF07388"]))
)

# cond4: neuS → remove if it contains neuA signatures
cond4 = (
    (df["gene"] == "NEUS") &
    df["extra_signatures"].apply(lambda x: has_inadequate_signature(x, ["G3DSA:3.90.550.10", "SSF53448"]))
)

# cond5: LIC3X, LST, PM0188 → remove neuA/neuS signatures
cond5 = (
    df["gene"].isin(["LIC3X", "LST", "PM0188"]) &
    df["extra_signatures"].apply(lambda x: has_inadequate_signature(x, ["PF07388", "G3DSA:3.90.550.10"]))
)

# === 4. Combine conditions and filter ===
cond_total = cond1 | cond2 | cond3 | cond4 | cond5

sequences_to_exclude = df[cond_total].copy()
df_filtered = df[~cond_total].copy()

# === 5. Save results ===
sequences_to_exclude.to_csv("sequences_to_exclude.csv", index=False)
df_filtered.to_csv("seqs_with_extra_signatures_filtered.csv", index=False)

# === 6. Report ===
print(f"🔹 Original rows: {len(df)}")
print(f"🔸 Sequences to exclude: {len(sequences_to_exclude)}")
print(f"✅ Remaining rows after curation: {len(df_filtered)}")

### Collecting IDs to be deleted (from sequences that have all signatures common to the references)

In [ ]:
# Load the dataframe
df = pd.read_csv("sequences_to_exclude.csv", sep=",") 

# Create output folder
os.makedirs("exclude", exist_ok=True)

# Iterate over unique genes
for gene, grupo in df.groupby("gene"):
    # Get all seq_id associated with this gene
    seq_ids = grupo["seq_id"].tolist()
    
    # Define the output file name
    output_file = f"exclude/{gene}_exclude_from_filtered_sequences.txt"
    
    # Save one ID per line
    with open(output_file, "w") as f:
        f.write("\n".join(seq_ids))
    
    print(f"File created: {output_file}")

In [ ]:
# Path to the folder where the files are located
pasta = "./exclude"  

# Pattern to find the files
files = glob.glob(os.path.join(pasta, "*.txt"))

# Output file path (in the same folder)
output_file = os.path.join(pasta, "concatenated.txt")

with open(output_file, "w") as output:
    for name in files:
        with open(name, "r") as f:
            content = f.read()
            output.write(content)
            output.write("\n")  

print(f"Concatenated file saved at:\n{output_file}")

The IDs to be deleted must be concatenated, and the code to remove them must search for all IDs in all files for the different genes, since they have been mixed together.

### Excluding inappropriate IDs from filtered sequences

In [ ]:
# === Folder paths and exclusion file ===
seq_folder = "filtered_sequences"
output_folder = "filtered_sequences_cleaned"
exclude_file = "./exclude/concatenated.txt"  # your concatenated file with all IDs

# === Create output folder ===
os.makedirs(output_folder, exist_ok=True)

# === Read IDs to exclude ===
with open(exclude_file) as f:
    ids_to_exclude = set(l.strip() for l in f if l.strip())

print(f"Total IDs to exclude: {len(ids_to_exclude)}")

# === Process all files in the folder ===
for file in os.listdir(seq_folder):
    if not file.endswith(".txt"):
        continue

    input_path = os.path.join(seq_folder, file)
    output_path = os.path.join(output_folder, file)

    with open(input_path) as f:
        lines = [l.strip() for l in f if l.strip()]

    # Remove any line whose ID is in the exclusion list
    filtered_lines = [l for l in lines if l not in ids_to_exclude]

    with open(output_path, "w") as f:
        f.write("\n".join(filtered_lines))

    print(f"Clean file saved: {output_path}")

print("\n✅ All files were processed successfully.")

### Extracting FASTA sequences for filtered clean IDs

The database FASTA files must be located in a subdirectory of this directory.

In [ ]:
# Folder paths
fasta_folder = "original_merged_fastas"
ids_folder = "filtered_sequences_cleaned"
output_folder = "filtered_fastas"

os.makedirs(output_folder, exist_ok=True)

# -------------------------------
# 1. Index all original sequences by ID
# -------------------------------
seq_dict = {}

for fasta_file in os.listdir(fasta_folder):
    if not fasta_file.endswith((".fasta", ".fa", ".faa")):
        continue

    fasta_path = os.path.join(fasta_folder, fasta_file)

    for seq in SeqIO.parse(fasta_path, "fasta"):
        seq_dict[seq.id] = seq  # ID -> SeqRecord object

print(f"✅ Total indexed sequences: {len(seq_dict)}")

# -------------------------------
# 2. Generate FASTAs per gene
# -------------------------------
for ids_file in os.listdir(ids_folder):
    if not ids_file.endswith(".txt"):
        continue

    gene = ids_file.split("_sequences")[0]
    ids_path = os.path.join(ids_folder, ids_file)

    # Read IDs to keep
    with open(ids_path) as f:
        kept_ids = [l.strip() for l in f if l.strip()]

    # Collect corresponding sequences
    filtered_seqs = [seq_dict[i] for i in kept_ids if i in seq_dict]

    # Output file
    output_fasta = os.path.join(output_folder, f"{gene}_filtered.fasta")

    # Write FASTA
    SeqIO.write(filtered_seqs, output_fasta, "fasta")

    print(f"✅ {gene}: {len(filtered_seqs)} sequences saved to {output_fasta}")